# 1. Packages

In [1]:
# PySpark packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, countDistinct, coalesce

In [2]:
spark = SparkSession.builder.appName("String").getOrCreate()

In [3]:
spark

# 2. The Dataset

In [4]:
loan = spark.read.csv('LoanStats_2018Q1.csv', inferSchema = True, header = True)

In [5]:
# 107868 records and 145 columns
print(loan.count())
print(len(loan.columns))

107868
145


In [6]:
# The goal is to predict the loan status for each loan application.
# 4 records were missing.
loan.filter((loan["loan_status"] == "") | loan["loan_status"].isNull() | isnan(loan["loan_status"])).\
select('loan_status').show()
print('{} records were missing under the loan_status column.'.format(
    loan.filter((loan["loan_status"] == "") | loan["loan_status"].isNull() | isnan(loan["loan_status"])).count())
     )

+-----------+
|loan_status|
+-----------+
|       null|
|       null|
|       null|
|       null|
+-----------+

4 records were missing under the loan_status column.


In [7]:
loan = loan.na.drop(subset = ["loan_status"])
print('Drop 4 missing records, now {} records left.'.format(loan.count()))

Drop 4 missing records, now 107864 records left.


# 3. Remove columns don't have prediction power
- **Columns have no prediction power can be columns:**
    1. Have only one value across all records. (E.g. url)
    2. Have unique values with the same length as the length of rows in raw data. (E.g. id)
    3. Have unique values with too many levels. (E.g. zip_code delivers the same information as addr_state.)
- Through the below observations, some columns have wrong data types. For instance, 'int_rate' and 'revol_util' has been incorrectly specified as string type.
- **Decision:** Columns including 'id', 'member_id', 'zip_code', 'policy_code' (observed from the metadata description), 'url', and 'desc' should be removed.

In [8]:
column_names = loan.columns
for col in column_names:
    unique_count = loan.select(countDistinct(col)).collect()
    if unique_count[0][0] == 0:
        print(col + ' : No unique value.')
    elif unique_count[0][0] == loan.count():
        print(col + ' : Each row is an unique value.')
    elif unique_count[0][0] > 10:
        if loan.select(col).dtypes[0][1] != 'double' and \
        loan.select(col).dtypes[0][1] != 'int':
            print(col + ' : More than 10 unique values.')
    else:
        continue

id : No unique value.
member_id : No unique value.
int_rate : More than 10 unique values.
sub_grade : More than 10 unique values.
emp_title : More than 10 unique values.
emp_length : More than 10 unique values.
url : No unique value.
desc : No unique value.
purpose : More than 10 unique values.
title : More than 10 unique values.
zip_code : More than 10 unique values.
addr_state : More than 10 unique values.
earliest_cr_line : More than 10 unique values.
revol_util : More than 10 unique values.
sec_app_earliest_cr_line : More than 10 unique values.


In [9]:
# Also, from the EDA part, 'title' and 'purpose' are similar features. 
# Now, 138 columns left.
loan = loan.drop('id','member_id','zip_code','policy_code','url','desc', 'title')
len(loan.columns)

138

# 4. Missing data imputation
## 4.1 Numerical features
- Deal with numerical features containing at least 80% NA values.
    - 4.1.1 Joint-type features
    - 4.1.2 Secondary applicant related features
    - 4.1.3 Hardship related features
    - 4.1.4 Settlement related features
    - 4.1.5 Other numerical features

In [10]:
column_names = loan.columns

In [11]:
i = 0
for col in column_names:
    if loan.select(col).dtypes != 'string':
        na_count = loan.filter((loan[col] == "") | loan[col].isNull() | isnan(loan[col])).count()
        if na_count >= loan.count()*0.8:
            print('Column: {} contained {} missing values.'.format(col, na_count))
            i += 1
print()
print(i, 'numerical type of features contain 80% of missing values.')

Column: mths_since_last_record contained 92595 missing values.
Column: annual_inc_joint contained 91533 missing values.
Column: dti_joint contained 91533 missing values.
Column: verification_status_joint contained 91847 missing values.
Column: mths_since_recent_bc_dlq contained 86566 missing values.
Column: revol_bal_joint contained 91533 missing values.
Column: sec_app_earliest_cr_line contained 91533 missing values.
Column: sec_app_inq_last_6mths contained 91533 missing values.
Column: sec_app_mort_acc contained 91533 missing values.
Column: sec_app_open_acc contained 91533 missing values.
Column: sec_app_revol_util contained 91843 missing values.
Column: sec_app_open_act_il contained 91533 missing values.
Column: sec_app_num_rev_accts contained 91533 missing values.
Column: sec_app_chargeoff_within_12_mths contained 91533 missing values.
Column: sec_app_collections_12_mths_ex_med contained 91533 missing values.
Column: sec_app_mths_since_last_major_derog contained 102437 missing val

### 4.1.1 Joint-type features
- Impute missing values under joint-type features by finding the corresponding not NA values from their paired columns. For instance, find not NA values under 'annual_inc' to replace NA values under 'annual_inc_joint'. Eventually, I will leave two columns as a pair, one tells the information for the 1st borrower, and the other one discloses the information for the 2nd borrower.
- Target columns: annual_inc_joint / dti_joint / verification_joint / revol_bal_joint

In [12]:
# 'application_type' indicated whether borrowers loaned as groups or individuals.
application_type = loan.select('application_type').collect()
i = 0
for each in application_type:
    if each[0] == 'Joint App':
        i += 1

In [13]:
# Use 'annual_inc' to impute missing values under 'annual_inc_joint'.
# 'annual_inc_joint'
print('{} applicants borrowed as groups.'.format(i))
print("{} records under 'annual_inc_joint' were not NULL.".format(
    loan.count() - loan.filter(loan["annual_inc_joint"].isNull()).count()))
print("{} records under 'annual_inc' were NULL.".format(
    loan.filter(loan["annual_inc"].isNull()).count()))

16331 applicants borrowed as groups.
16331 records under 'annual_inc_joint' were not NULL.
0 records under 'annual_inc' were NULL.


In [14]:
# 'dti_joint'
print("{} records under 'dti_joint' were not NULL.".format(
    loan.count() - loan.filter(loan["dti_joint"].isNull()).count()))
print("{} records under 'dti' were NULL.".format(
    loan.filter(loan["dti"].isNull()).count()))

16331 records under 'dti_joint' were not NULL.
262 records under 'dti' were NULL.


In [15]:
# 'verification_status_joint'
print("{} records under 'verification_status_joint' were not NULL.".format(
    loan.count() - loan.filter(loan["verification_status_joint"].isNull()).count()))
print("{} records under 'verification_status' were NULL.".format(
    loan.filter(loan["verification_status"].isNull()).count()))

16017 records under 'verification_status_joint' were not NULL.
0 records under 'verification_status' were NULL.


In [16]:
# 'revol_bal_joint'
print("{} records under 'revol_bal_joint' were not NULL.".format(
    loan.count() - loan.filter(loan["revol_bal_joint"].isNull()).count()))
print("{} records under 'revol_bal' were NULL.".format(
    loan.filter(loan["revol_bal"].isNull()).count()))

16331 records under 'revol_bal_joint' were not NULL.
0 records under 'revol_bal' were NULL.


In [17]:
# Joint-type features missing data imputation
loan = loan.withColumn("annual_inc_joint", coalesce(loan.annual_inc_joint,loan.annual_inc))
loan = loan.withColumn("dti_joint", coalesce(loan.dti_joint,loan.dti))
loan = loan.withColumn("verification_status_joint", coalesce(loan.verification_status_joint,loan.verification_status))
loan = loan.withColumn("revol_bal_joint", coalesce(loan.revol_bal_joint,loan.revol_bal))

In [18]:
# Make sure the missing data imputation is correct 
# by checking there is no missing data anymore.
print(loan.filter(loan['annual_inc_joint'].isNull()).count())
print(loan.filter(loan['dti_joint'].isNull()).count())
print(loan.filter(loan['verification_status_joint'].isNull()).count())
print(loan.filter(loan['revol_bal_joint'].isNull()).count())

0
0
0
0


In [19]:
# Check how many columns.
print(len(loan.columns))

138


### 4.1.2 Secondary applicant related features
- Impute missing values under secondary applicant related features by replacing '0', since those missing records came from loan applicants who loaned individually after reading the metadata description.
- Target columns: sec_app_earliest_cr_line / sec_app_inq_last_6mths / sec_app_mort_acc / sec_app_open_acc / sec_app_revol_util / sec_app_open_act_il / sec_app_num_rev_accts / sec_app_chargeoff_within_12_mths / sec_app_collections_12_mths_ex_med / sec_app_mths_since_last_major_derog

In [20]:
target_columns = ['sec_app_earliest_cr_line','sec_app_inq_last_6mths',
                  'sec_app_mort_acc','sec_app_open_acc', 
                  'sec_app_revol_util','sec_app_open_act_il',
                  'sec_app_num_rev_accts','sec_app_chargeoff_within_12_mths',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_mths_since_last_major_derog']

paired_columns = ['earliest_cr_line','inq_last_6mths',
                  'mort_acc','open_acc',
                  'revol_util','open_act_il',
                  'num_rev_accts','chargeoff_within_12_mths',
                  'collections_12_mths_ex_med',
                  'mths_since_last_major_derog']                

In [21]:
# Check there is no missing value from the paired column such that the missing value
# imputation can be done successfully.
for i in range(len(target_columns)):
    print("{} records under ".format(
        loan.count() - loan.filter(loan[target_columns[i]].isNull()).count())
         +target_columns[i]+ " were not NULL.")
    print("{} records under ".format(
        loan.filter(loan[paired_columns[i]].isNull()).count())
         +paired_columns[i]+ " were NULL.")
    print('-'*60)

16331 records under sec_app_earliest_cr_line were not NULL.
0 records under earliest_cr_line were NULL.
------------------------------------------------------------
16331 records under sec_app_inq_last_6mths were not NULL.
0 records under inq_last_6mths were NULL.
------------------------------------------------------------
16331 records under sec_app_mort_acc were not NULL.
0 records under mort_acc were NULL.
------------------------------------------------------------
16331 records under sec_app_open_acc were not NULL.
0 records under open_acc were NULL.
------------------------------------------------------------
16021 records under sec_app_revol_util were not NULL.
149 records under revol_util were NULL.
------------------------------------------------------------
16331 records under sec_app_open_act_il were not NULL.
0 records under open_act_il were NULL.
------------------------------------------------------------
16331 records under sec_app_num_rev_accts were not NULL.
0 records

In [22]:
# Make sure the missing data imputation is correct 
# by checking there is no missing data anymore.
# Still two columns have missing data. 
for i in range(len(target_columns)):
    loan = loan.withColumn(target_columns[i], 
                           coalesce(loan[target_columns[i]],
                                    loan[paired_columns[i]]))
    print(loan.filter(loan[target_columns[i]].isNull()).count())

0
0
0
0
109
0
0
0
0
79728


In [23]:
target_columns[4], target_columns[-1]

('sec_app_revol_util', 'sec_app_mths_since_last_major_derog')

### 4.1.3 Hardship related features
- Only one borrower (record) used the Hardship. From a modeling perspective, there are only two levels for each hardship feature, and thus has trivial prediction power on the loan_status variable.
- Target columns: hardship_type / hardship_reason / hardship_status / hardship_amount / hardship_start_date / hardship_end_date / hardship_length / hardship_dpd / hardship_loan_status / hardship_payoff_balance_amount / hardship_last_payment_amount / hardship_flag / deferral_term / payment_plan_start_date / orig_projected_additional_accrued_interest /

In [24]:
target_columns = ['hardship_type','hardship_reason','hardship_status',
                  'hardship_amount','hardship_start_date','hardship_end_date',
                  'hardship_length','hardship_dpd','hardship_loan_status',
                  'hardship_payoff_balance_amount','hardship_last_payment_amount',
                  'hardship_flag',
                  'deferral_term','payment_plan_start_date','orig_projected_additional_accrued_interest']

# There is only one record available
loan.filter(~loan['hardship_type'].isNull()).select(target_columns).show()

+--------------------+---------------+---------------+---------------+-------------------+-----------------+---------------+------------+--------------------+------------------------------+----------------------------+-------------+-------------+-----------------------+------------------------------------------+
|       hardship_type|hardship_reason|hardship_status|hardship_amount|hardship_start_date|hardship_end_date|hardship_length|hardship_dpd|hardship_loan_status|hardship_payoff_balance_amount|hardship_last_payment_amount|hardship_flag|deferral_term|payment_plan_start_date|orig_projected_additional_accrued_interest|
+--------------------+---------------+---------------+---------------+-------------------+-----------------+---------------+------------+--------------------+------------------------------+----------------------------+-------------+-------------+-----------------------+------------------------------------------+
|INTEREST ONLY-3 M...|   UNEMPLOYMENT|         ACTIVE|    

In [25]:
# Drop off all the hardship related features since they only contribute a litte 
# prediction power on the 'loan_status'.
print('Drop off {} columns from the dataframe.'.format(len(target_columns)))

Drop off 15 columns from the dataframe.


In [26]:
loan = loan.drop('hardship_type','hardship_reason','hardship_status',
                 'hardship_amount','hardship_start_date','hardship_end_date',
                 'hardship_length','hardship_dpd','hardship_loan_status',
                 'hardship_payoff_balance_amount','hardship_last_payment_amount',
                 'hardship_flag',
                 'deferral_term','payment_plan_start_date','orig_projected_additional_accrued_interest'
                )

In [27]:
# 124 columns left in the dataset.
len(loan.columns)

123

### 4.1.4 Settlement related features
- Only nine borrowers (records) used the Settlement, which means that they were working with a debt-settlement company. From a modeling perspective, settlement related features have little contribution on the loan_status variable.
- Target columns: debt_settlement_flag_date / settlement_status / settlement_date / settlement_amount / settlement_percentage / settlement_term / debt_settlement_flag

In [28]:
target_columns = ['debt_settlement_flag_date','settlement_status',
                  'settlement_date','settlement_amount',
                  'settlement_percentage','settlement_term',
                  'debt_settlement_flag']
len(target_columns)

7

In [29]:
# There is only one record available
loan.filter(~loan['debt_settlement_flag_date'].isNull()).select(target_columns).show()

+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+--------------------+
|debt_settlement_flag_date|settlement_status|settlement_date|settlement_amount|settlement_percentage|settlement_term|debt_settlement_flag|
+-------------------------+-----------------+---------------+-----------------+---------------------+---------------+--------------------+
|                   Jul-18|           ACTIVE|         Jul-18|             1744|                65.01|             17|                   Y|
|                   Jun-18|           ACTIVE|         Jun-18|             5018|                65.01|             10|                   Y|
|                   Jul-18|           ACTIVE|         Jul-18|             4018|                65.01|             16|                   Y|
|                   Jul-18|           ACTIVE|         Jul-18|             1854|                64.99|             12|                   Y|
|                   Jun-18|

In [30]:
# Drop off all the settlement related features since they only contribute a litte 
# prediction power on the 'loan_status'.
print('Drop off {} columns from the dataframe.'.format(len(target_columns)))

Drop off 7 columns from the dataframe.


In [31]:
loan = loan.drop('debt_settlement_flag_date','settlement_status',
                  'settlement_date','settlement_amount',
                  'settlement_percentage','settlement_term',
                  'debt_settlement_flag'
                )

In [32]:
# 117 columns left in the dataset.
len(loan.columns)

116

### 4.1.5 Other features

In [33]:
column_names = loan.columns

In [34]:
# Find the rest of features which still have missing records.
numeric_feat = {}
string_feat = {}

i = 0
for col in column_names:
    na_count = loan.filter((loan[col] == "") | loan[col].isNull() | isnan(loan[col])).count()
    if na_count != 0:
        print('Column: {} contained {} missing values.'.format(col, na_count))
        i += 1
        if loan.select(col).dtypes[0][1] != 'string':
            numeric_feat[col] = na_count
        elif loan.select(col).dtypes[0][1] == 'string':
            string_feat[col] = na_count
print()
print(i, 'features still contain missing values.')

Column: emp_title contained 9613 missing values.
Column: dti contained 262 missing values.
Column: mths_since_last_delinq contained 60695 missing values.
Column: mths_since_last_record contained 92595 missing values.
Column: revol_util contained 149 missing values.
Column: last_pymnt_d contained 151 missing values.
Column: next_pymnt_d contained 6517 missing values.
Column: last_credit_pull_d contained 2 missing values.
Column: mths_since_last_major_derog contained 83103 missing values.
Column: mths_since_rcnt_il contained 4288 missing values.
Column: il_util contained 17984 missing values.
Column: all_util contained 28 missing values.
Column: avg_cur_bal contained 10 missing values.
Column: bc_open_to_buy contained 1510 missing values.
Column: bc_util contained 1561 missing values.
Column: mo_sin_old_il_acct contained 4288 missing values.
Column: mths_since_recent_bc contained 1421 missing values.
Column: mths_since_recent_bc_dlq contained 86566 missing values.
Column: mths_since_rece

In [35]:
len(numeric_feat), len(string_feat)

(19, 6)

In [36]:
# Save the current dataset into a csv for the next step.
loan.toPandas().to_csv('loan.csv')